# 🚀 kangtani.ai - Agricultural Chatbot Assistant

This notebook runs the complete kangtani.ai project (backend + frontend) in Google Colab with ngrok port forwarding.

## What this does:
- Clones the GitHub repository
- Installs all dependencies
- Runs Ollama with Gemma model
- Starts FastAPI backend
- Starts Next.js frontend
- Sets up ngrok for public access

## Prerequisites:
- GitHub repository URL (you'll need to provide this)
- ngrok authtoken (optional, for custom domains)

## 📋 Setup & Installation

In [ ]:
# Install system dependencies
!apt-get update
!apt-get install -y curl wget git build-essential

# Install Node.js and npm
!curl -fsSL https://deb.nodesource.com/setup_18.x | bash -
!apt-get install -y nodejs

# Install pnpm
!npm install -g pnpm

# Install Ollama
!curl -fsSL https://ollama.ai/install.sh | sh

# Install ngrok
!wget https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.tgz
!tar xvzf ngrok-v3-stable-linux-amd64.tgz
!mv ngrok /usr/local/bin/

# Install Python dependencies for backend
!pip install fastapi uvicorn[standard] httpx python-multipart
!pip install openai-whisper PyPDF2 pdfplumber python-docx

print("✅ All dependencies installed successfully!")

In [ ]:
# Clone the GitHub repository
# Replace with your actual GitHub repository URL
GITHUB_REPO = "https://github.com/yourusername/kangtani.ai.git"

# You can also use a direct download if the repo is public
# GITHUB_REPO = "https://github.com/yourusername/kangtani.ai/archive/refs/heads/main.zip"

print(f"📥 Cloning repository: {GITHUB_REPO}")

if GITHUB_REPO.endswith('.git'):
    !git clone {GITHUB_REPO}
    !cd kangtani.ai && ls -la
else:
    !wget {GITHUB_REPO} -O kangtani.zip
    !unzip kangtani.zip
    !ls -la

print("✅ Repository cloned successfully!")

In [ ]:
# Setup ngrok (optional - for custom domains)
# If you have an ngrok authtoken, uncomment and add it below
# NGROK_AUTHTOKEN = "your_ngrok_authtoken_here"
# !ngrok config add-authtoken {NGROK_AUTHTOKEN}

print("🔧 ngrok configured (using free tier)")

## 🤖 Ollama Setup & Model Download

In [ ]:
# Start Ollama service in background
import subprocess
import time
import requests

print("🚀 Starting Ollama service...")

# Start Ollama in background
ollama_process = subprocess.Popen(["ollama", "serve"], 
                                  stdout=subprocess.PIPE, 
                                  stderr=subprocess.PIPE)

# Wait for Ollama to start
time.sleep(5)

# Test Ollama connection
try:
    response = requests.get("http://localhost:11434/api/tags")
    if response.status_code == 200:
        print("✅ Ollama service is running!")
    else:
        print("⚠️  Ollama service might not be ready yet...")
except:
    print("⚠️  Ollama service starting...")

print("📥 Downloading Gemma model (this may take a while)...")
!ollama pull gemma

print("✅ Gemma model downloaded successfully!")

## 🔧 Backend Setup & Installation

In [ ]:
# Navigate to backend directory and install dependencies
!cd kangtani.ai/backend && pip install -r requirements.txt

print("✅ Backend dependencies installed!")

In [ ]:
# Start FastAPI backend in background
import subprocess
import time
import requests

print("🚀 Starting FastAPI backend...")

# Start backend in background
backend_process = subprocess.Popen([
    "uvicorn", "main:app", "--host", "0.0.0.0", "--port", "8000"
], cwd="kangtani.ai/backend", 
   stdout=subprocess.PIPE, 
   stderr=subprocess.PIPE)

# Wait for backend to start
time.sleep(10)

# Test backend connection
try:
    response = requests.get("http://localhost:8000/health")
    if response.status_code == 200:
        print("✅ Backend is running!")
        print(f"📊 Health check: {response.json()}")
    else:
        print("⚠️  Backend might not be ready yet...")
except Exception as e:
    print(f"⚠️  Backend starting... Error: {e}")

print("🌐 Backend URL: http://localhost:8000")

## 🎨 Frontend Setup & Installation

In [ ]:
# Navigate to frontend directory and install dependencies
!cd kangtani.ai/frontend && pnpm install

print("✅ Frontend dependencies installed!")

In [ ]:
# Start Next.js frontend in background
import subprocess
import time
import requests

print("🚀 Starting Next.js frontend...")

# Start frontend in background
frontend_process = subprocess.Popen([
    "pnpm", "dev", "--hostname", "0.0.0.0", "--port", "3000"
], cwd="kangtani.ai/frontend", 
   stdout=subprocess.PIPE, 
   stderr=subprocess.PIPE)

# Wait for frontend to start
time.sleep(15)

# Test frontend connection
try:
    response = requests.get("http://localhost:3000")
    if response.status_code == 200:
        print("✅ Frontend is running!")
    else:
        print("⚠️  Frontend might not be ready yet...")
except Exception as e:
    print(f"⚠️  Frontend starting... Error: {e}")

print("🌐 Frontend URL: http://localhost:3000")

## 🌍 ngrok Port Forwarding Setup

In [ ]:
# Start ngrok for public access
import subprocess
import time
import requests
import json

print("🌍 Setting up ngrok port forwarding...")

# Start ngrok for frontend (port 3000)
ngrok_frontend = subprocess.Popen([
    "ngrok", "http", "3000"
], stdout=subprocess.PIPE, stderr=subprocess.PIPE)

# Wait for ngrok to start
time.sleep(5)

# Get ngrok public URL
try:
    response = requests.get("http://localhost:4040/api/tunnels")
    if response.status_code == 200:
        tunnels = response.json()
        if tunnels['tunnels']:
            public_url = tunnels['tunnels'][0]['public_url']
            print(f"✅ ngrok is running!")
            print(f"🌐 Public URL: {public_url}")
            print(f"🔗 ngrok dashboard: http://localhost:4040")
        else:
            print("⚠️  ngrok tunnel not ready yet...")
    else:
        print("⚠️  ngrok API not accessible...")
except Exception as e:
    print(f"⚠️  ngrok starting... Error: {e}")

print("\n📝 Note: The frontend will be accessible via the ngrok URL above.")
print("📝 The backend API will be available at: {public_url.replace('https://', 'https://').replace('http://', 'https://').replace(':3000', ':8000')}")
print("📝 You may need to update the frontend API URL to use the ngrok backend URL.")

## 📊 Status Check

In [ ]:
# Check status of all services
import requests
import subprocess

print("🔍 Checking service status...")
print("=" * 50)

# Check Ollama
try:
    response = requests.get("http://localhost:11434/api/tags")
    if response.status_code == 200:
        print("✅ Ollama: Running")
        models = response.json().get('models', [])
        if models:
            print(f"   📦 Models: {', '.join([m['name'] for m in models])}")
    else:
        print("❌ Ollama: Not responding")
except:
    print("❌ Ollama: Not accessible")

# Check Backend
try:
    response = requests.get("http://localhost:8000/health")
    if response.status_code == 200:
        print("✅ Backend: Running")
        health = response.json()
        print(f"   🔗 Health: {health.get('status', 'unknown')}")
    else:
        print("❌ Backend: Not responding")
except:
    print("❌ Backend: Not accessible")

# Check Frontend
try:
    response = requests.get("http://localhost:3000")
    if response.status_code == 200:
        print("✅ Frontend: Running")
    else:
        print("❌ Frontend: Not responding")
except:
    print("❌ Frontend: Not accessible")

# Check ngrok
try:
    response = requests.get("http://localhost:4040/api/tunnels")
    if response.status_code == 200:
        tunnels = response.json()
        if tunnels['tunnels']:
            print("✅ ngrok: Running")
            print(f"   🌐 Public URL: {tunnels['tunnels'][0]['public_url']}")
        else:
            print("❌ ngrok: No tunnels")
    else:
        print("❌ ngrok: Not responding")
except:
    print("❌ ngrok: Not accessible")

print("=" * 50)
print("🎉 Setup complete! Your kangtani.ai is now running.")

## 🎯 Usage Instructions

1. **Access the application**: Use the ngrok public URL provided above
2. **Start chatting**: The chatbot will respond using the Gemma model via Ollama
3. **Monitor logs**: Check the ngrok dashboard at `http://localhost:4040`
4. **Stop services**: Run the cell below to stop all services

### API Endpoints:
- Frontend: `{ngrok_url}`
- Backend API: `{ngrok_url.replace(':3000', ':8000')}`
- Health Check: `{ngrok_url.replace(':3000', ':8000')}/health`
- Chat API: `{ngrok_url.replace(':3000', ':8000')}/chat`

In [ ]:
# Stop all services
import subprocess
import signal
import os

print("🛑 Stopping all services...")

# Kill all processes
try:
    subprocess.run(["pkill", "-f", "ollama"], check=False)
    subprocess.run(["pkill", "-f", "uvicorn"], check=False)
    subprocess.run(["pkill", "-f", "next"], check=False)
    subprocess.run(["pkill", "-f", "ngrok"], check=False)
    print("✅ All services stopped!")
except Exception as e:
    print(f"⚠️  Error stopping services: {e}")

print("\n📝 Note: You can restart the services by running the cells above again.")

## 🔧 Troubleshooting

### Common Issues:

1. **Ollama not starting**:
   - Check if Ollama is installed correctly
   - Try running `!ollama serve` manually

2. **Model download fails**:
   - Check internet connection
   - Try downloading a smaller model: `!ollama pull gemma:2b`

3. **Port conflicts**:
   - Change ports in the startup commands
   - Check if ports are already in use

4. **ngrok not working**:
   - Check ngrok dashboard at `http://localhost:4040`
   - Restart ngrok if needed

5. **Frontend can't connect to backend**:
   - Update the API URL in `frontend/hooks/useChat.ts`
   - Use the ngrok backend URL instead of localhost

### Manual Commands:
```bash
# Start Ollama
ollama serve

# Start Backend
cd kangtani.ai/backend && uvicorn main:app --host 0.0.0.0 --port 8000

# Start Frontend
cd kangtani.ai/frontend && pnpm dev --hostname 0.0.0.0 --port 3000

# Start ngrok
ngrok http 3000
```